# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# Import pprint
from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
clean_weather_df = pd.read_csv('../WeatherPy/weather_data.csv')
clean_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,hoi an,15.8794,108.3350,94.96,59.0,100,12.21,VN,1628046480
1,ubinskoye,55.3068,79.6802,61.97,79.0,96,4.43,RU,1628046480
2,paamiut,61.9940,-49.6678,45.23,95.0,100,5.17,GL,1628046481
3,norman wells,65.2820,-126.8329,55.27,76.0,100,2.33,CA,1628046481
4,punta arenas,-53.1500,-70.9167,42.12,75.0,90,9.22,CL,1628046482


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Create variables to store data
locations = clean_weather_df[['Lat', 'Lng']]
humidity = clean_weather_df['Humidity']

# Configure gmap
gmaps.configure(api_key=g_key)
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, point_radius=2,
                                 max_intensity=100)
                                 
# # Add heat map layer to fig with .add_layer() function
fig.add_layer(heat_layer)

# # Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the DataFrame to find my ideal weather condition
personal_df = clean_weather_df[(clean_weather_df['Max Temp'] > 60) & 
              (clean_weather_df['Max Temp'] < 90) & 
              (clean_weather_df['Humidity'] < 60) & 
              (clean_weather_df['Cloudiness'] < 10) &
              (clean_weather_df['Wind Speed'] < 10)]
personal_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,boca do acre,-8.7522,-67.3978,71.10,50.0,0,1.70,BR,1628046483
66,sierra vista,31.5545,-110.3037,84.67,45.0,1,9.22,US,1628046502
67,conceicao do araguaia,-8.2578,-49.2647,70.41,40.0,5,3.27,BR,1628046503
71,portland,45.5234,-122.6762,86.45,42.0,0,1.01,US,1628045714
79,sedro-woolley,48.5039,-122.2361,75.94,59.0,1,6.91,US,1628046506


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
# Store the df as hotel_df
hotel_df = personal_df

# Set parameters to search for hotels with 5000 meters.
params = {
    'key': g_key,
    'radius': 5000,
    'types': 'lodging',
    'keyword': 'hotel'
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Add base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Complete the url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # Print the results
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing result... skipping.")
        
    print("--------------------------")

Retrieving Results for Index 9: boca do acre.
Closest hotel is Hotel Alice.
--------------------------
Retrieving Results for Index 66: sierra vista.


/Users/yankori/Desktop/Local/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/Users/yankori/Desktop/Local/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Closest hotel is Gateway Studio Suites.
--------------------------
Retrieving Results for Index 67: conceicao do araguaia.
Closest hotel is Tarumã Tropical Hotel.
--------------------------
Retrieving Results for Index 71: portland.
Closest hotel is Kimpton RiverPlace Hotel.
--------------------------
Retrieving Results for Index 79: sedro-woolley.
Closest hotel is Three Rivers Inn Hotel.
--------------------------
Retrieving Results for Index 86: kasongo-lunda.
Missing result... skipping.
--------------------------
Retrieving Results for Index 111: aklavik.
Closest hotel is Sundog Inn.
--------------------------
Retrieving Results for Index 113: urumqi.
Closest hotel is Sheraton Urumqi Hotel.
--------------------------
Retrieving Results for Index 134: komsomolskiy.
Closest hotel is TANTANA HOTEL FERGANA.
--------------------------
Retrieving Results for Index 139: turukhansk.
Closest hotel is ООО "Туруханская пушнина".
--------------------------
Retrieving Results for Index 166: esik

In [9]:
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,boca do acre,-8.7522,-67.3978,71.10,50.0,0,1.70,BR,1628046483,Hotel Alice
66,sierra vista,31.5545,-110.3037,84.67,45.0,1,9.22,US,1628046502,Gateway Studio Suites
67,conceicao do araguaia,-8.2578,-49.2647,70.41,40.0,5,3.27,BR,1628046503,Tarumã Tropical Hotel
71,portland,45.5234,-122.6762,86.45,42.0,0,1.01,US,1628045714,Kimpton RiverPlace Hotel
79,sedro-woolley,48.5039,-122.2361,75.94,59.0,1,6.91,US,1628046506,Three Rivers Inn Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer on top of heat map
hotel_layer = gmaps.symbol_layer(
    locations, 
    scale=2,
    info_box_content=hotel_info
)

# Display figure
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))